## Sentiment Signals

In [64]:
# Import libraries
import os
from pathlib import Path
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
load_dotenv()
import tweepy
import time
print("Libraries imported")

Libraries imported


### Data download, Vader sentiment, Trading signals

In [37]:
#Setup tools for downloading newsapi and vader sentiment analysis
api_key = os.getenv("news_api")
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key=api_key)
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\khuli\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [86]:
# Loop to get the news topics, generate sentiment score and set the signal from sentiment score
# Dealing with multiple articles within the same day, pivot the dataframe to get average of the compund score 
# If average sentiment score > 0.5 then signal = 1 (Buy), score < -0.5 then signal = -1 (Sell), otherwise signal = 0 (Hold)
upperbound = 0.5
lowerbound = -0.5

def sentiment(query):
    # Reading the News API key enviroment variable   
    headlines = newsapi.get_everything(q=query,language="en",page_size=100,sort_by="relevancy")
    # Create the sentiment scores DataFrame
    sentiments = []
    for article in headlines["articles"]:
        try:
            date = article["publishedAt"][:10]
            text = article["content"]
            sentiment = analyzer.polarity_scores(text)
            compound = sentiment["compound"]
            sentiments.append({"date": date,"compound": compound})
        except AttributeError:
            pass
    sentiments = pd.DataFrame(sentiments)
    sentiments.set_index('date', inplace = True)
    sentiments = sentiments.pivot_table(index="date",aggfunc = {"compound": np.mean})
    sentiments["buy"] = np.where(sentiments["compound"] > upperbound, 1.0, 0.0)
    sentiments["sell"] = np.where(sentiments["compound"] < lowerbound, -1.0, 0.0)
    sentiments["signal"] = sentiments["buy"] + sentiments["sell"] 
    sentiments = sentiments.drop(columns = ["buy", "sell", "compound"])
    return sentiments


In [87]:
# Test the sentiment function
boeing = sentiment("boeing")
boeing

,signal
date,
2020-09-12,0.0
2020-09-13,0.0
2020-09-15,-1.0
2020-09-16,0.0
2020-09-17,0.0
2020-09-20,0.0
2020-09-21,0.0
2020-09-22,0.0
2020-09-24,0.0


### Create sentiments trading signal for DJ30

In [88]:
# Read company list
company_list = pd.read_csv("../Project2/company_list_DJ30.csv")
company_list.drop('Name', axis=1, inplace=True)
company_list["Keyword"]= company_list["Keyword"].astype(str) 
company_list.tail()

,Ticker,Keyword
25,V,"""Visa"""
26,WBA,"""Walgreens"""
27,WMT,"""Walmart"""
28,DIS,"""Disney"""
29,DOW,"""Dow"""


In [92]:
# Create sentiment scores dataframe for each stock in DJ30 by looping into company_list df
sentiments = pd.DataFrame()
L = []
count = 0
for t, k in zip(company_list.Ticker,company_list.Keyword):
    count += 1
    L = sentiment(k)
    sentiments = pd.concat([sentiments,L], axis=1, join='outer')
    sentiments.fillna(0, inplace = True)
sentiments.columns= ["AXP","AMGN","AAPL","BA","CAT","CSCO","CVX","GS","HD","HON","IBM","INTC","JNJ","KO","JPM","MCD","MMM","MRK","MSFT","NKE","PG","TRV","UNH","CRM","VZ","V","WBA","WMT","DIS","DOW"
]

C:\Users\khuli\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [93]:
sentiments

,AXP,AMGN,AAPL,BA,CAT,CSCO,CVX,GS,HD,HON,...,PG,TRV,UNH,CRM,VZ,V,WBA,WMT,DIS,DOW
2020-09-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-09-13,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-09-14,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2020-09-15,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2020-09-16,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-09-17,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-09-18,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-09-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2020-09-20,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2020-09-21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Export csv of sentiments signals

In [95]:
sentiments.to_csv(r'../Project2/sentiment_signal.csv')